In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, classification_report

from xgboost import XGBClassifier

import joblib


In [45]:
df = pd.read_csv("Machine-Learning.csv", low_memory=False)

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

df = df.drop_duplicates()

print(df.shape)
df.head()


(102956, 28)


,incident_date,incident_year,incident_month,incident_day,operator,aircraft,aircraft_type,aircraft_make,aircraft_model,aircraft_mass,...,timeofday,precipitation,height,distance,species_name,flight_impact,aircraft_damage,engine_ingested,visibility,season
0,2000-01-01,2000,1,1,UNITED AIRLINES,B-737-300,A,148,24.0,4.0,...,NaN,NaN,NaN,NaN,UNKNOWN MEDIUM BIRD,NaN,No,0,Clear,Winter
1,2000-01-01,2000,1,1,AMERICAN AIRLINES,B-727-200,A,148,11.0,4.0,...,DAWN,NONE,0.0,0.0,UNKNOWN SMALL BIRD,NONE,No,0,Clear,Winter
2,2000-01-01,2000,1,1,CONTINENTAL AIRLINES,B-757-200,A,148,26.0,4.0,...,DAY,FOG,0.0,0.0,UNKNOWN MEDIUM BIRD,NaN,No,0,Poor,Winter
3,2000-01-01,2000,1,1,US CUSTOMS AND BORDER PROTECTION,C-550,A,226,37.0,3.0,...,DAY,NaN,1000.0,NaN,UNKNOWN LARGE BIRD,PRECAUTIONARY LANDING,Yes,0,Clear,Winter
4,2000-01-01,2000,1,1,UNITED AIRLINES,B-727-200,A,148,11.0,4.0,...,NaN,NaN,NaN,NaN,UNKNOWN MEDIUM BIRD,NaN,No,0,Clear,Winter


In [46]:

df["visibility"] = df["visibility"].astype(str)
df["airport"] = df["airport"].astype(str)
df["timeofday"] = df["timeofday"].astype(str)


In [47]:

df["aircraft_damage"] = df["aircraft_damage"].map({"Yes": 1, "No": 0})

df = df.dropna(subset=["aircraft_damage"])

features = [
    "incident_year",
    "incident_month",
    "timeofday",
    "airport",
    "visibility"
]

num_features = ["incident_year", "incident_month"]
cat_features = ["timeofday", "airport", "visibility"]

X = df[features]
y = df["aircraft_damage"]


In [48]:
preprocessor = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_features),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_features)
])


In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [50]:
xgb_birdstrike = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

pipe_birdstrike = Pipeline([
    ("preprocessor", preprocessor),
    ("model", xgb_birdstrike)
])

param_grid_birdstrike = {
    "model__n_estimators": [200, 300],
    "model__max_depth": [4, 6],
    "model__learning_rate": [0.05, 0.1],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0]
}

grid_birdstrike = GridSearchCV(
    pipe_birdstrike,
    param_grid_birdstrike,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=2
)

grid_birdstrike.fit(X_train, y_train)

best_birdstrike_model = grid_birdstrike.best_estimator_


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=0.8; total time=   0.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=1.0; total time=   0.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=1.0; total time=   0.6s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=1.0; total time=   0.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=0.8; total time=   0.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200, model__subsample=0.8; total time=   0.7s
[CV] END model__colsample_bytree=

In [51]:
y_prob = best_birdstrike_model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


ROC-AUC: 0.6878794993185425


In [52]:
strike_df = df[df["aircraft_damage"] == 1]


In [53]:
from sklearn.preprocessing import LabelEncoder

# Drop missing values
species_df = strike_df.dropna(subset=features + ["species_name"]).copy()
species_df["species_name"] = species_df["species_name"].astype(str)

# Keep species with at least 20 occurrences
MIN_COUNT = 20
species_counts = species_df["species_name"].value_counts()

species_df["species_name_grouped"] = species_df["species_name"].apply(
    lambda x: x if species_counts[x] >= MIN_COUNT else "Other"
)


In [54]:
species_encoder = LabelEncoder()

X_species = species_df[features]
y_species = species_encoder.fit_transform(species_df["species_name_grouped"])


In [55]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_species,
    y_species,
    test_size=0.2,
    random_state=42,
    stratify=y_species
)


In [56]:
xgb_species = XGBClassifier(
    objective="multi:softprob",
    eval_metric="mlogloss",
    random_state=42,
    num_class=len(np.unique(y_species))
)

pipe_species = Pipeline([
    ("preprocessor", preprocessor),
    ("model", xgb_species)
])

param_grid_species = {
    "model__n_estimators": [200, 300],
    "model__max_depth": [4, 6],
    "model__learning_rate": [0.05, 0.1]
}

grid_species = GridSearchCV(
    pipe_species,
    param_grid_species,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)

grid_species.fit(X_train_s, y_train_s)

best_species_model = grid_species.best_estimator_


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   6.3s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   6.5s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   6.8s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   9.5s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   9.6s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   9.6s
[CV] END model__learning_rate=0.05, model__max_depth=6, model__n_estimators=200; total time=  10.0s
[CV] END model__learning_rate=0.05, model__max_depth=6, model__n_estimators=200; total time=  10.0s
[CV] END model__learning_rate=0.1, model__max_depth=4, model__n_estimators=200; total time=   6.2s
[CV] END model__learning_rate=0.1, model_

In [57]:
phase_df = strike_df.dropna(subset=features + ["flight_phase"]).copy()
phase_df["flight_phase"] = phase_df["flight_phase"].astype(str)

from sklearn.preprocessing import LabelEncoder
phase_encoder = LabelEncoder()

X_phase = phase_df[features]
y_phase = phase_encoder.fit_transform(phase_df["flight_phase"])



In [58]:



phase_df = strike_df.dropna(subset=features + ["flight_phase"]).copy()


phase_df["flight_phase"] = phase_df["flight_phase"].astype(str)


phase_encoder = LabelEncoder()

X_phase = phase_df[features]
y_phase = phase_encoder.fit_transform(phase_df["flight_phase"])


X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_phase,
    y_phase,
    test_size=0.2,
    random_state=42,
    stratify=y_phase
)


xgb_phase = XGBClassifier(
    objective="multi:softprob",
    eval_metric="mlogloss",
    random_state=42,
    num_class=len(np.unique(y_phase))
)


pipe_phase = Pipeline([
    ("preprocessor", preprocessor),
    ("model", xgb_phase)
])


param_grid_phase = {
    "model__n_estimators": [200, 300],
    "model__max_depth": [4, 6],
    "model__learning_rate": [0.05, 0.1]
}

grid_phase = GridSearchCV(
    pipe_phase,
    param_grid_phase,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)


grid_phase.fit(X_train_p, y_train_p)


best_phase_model = grid_phase.best_estimator_


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   1.6s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   1.6s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=200; total time=   1.7s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   2.4s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   2.4s
[CV] END model__learning_rate=0.05, model__max_depth=4, model__n_estimators=300; total time=   2.4s
[CV] END model__learning_rate=0.05, model__max_depth=6, model__n_estimators=200; total time=   2.6s
[CV] END model__learning_rate=0.05, model__max_depth=6, model__n_estimators=200; total time=   2.6s
[CV] END model__learning_rate=0.05, model__max_depth=6, model__n_estimators=200; total time=   2.5s
[CV] END model__learning_rate=0.1, model

In [59]:
pd.Series(phase_encoder.classes_)


0         APPROACH
1          ARRIVAL
2            CLIMB
3        DEPARTURE
4          DESCENT
5         EN ROUTE
6          LANDING
7     LANDING ROLL
8            LOCAL
9              NAN
10          PARKED
11     TAKEOFF RUN
12            TAXI
dtype: object

In [60]:
joblib.dump(best_birdstrike_model, "xgb_birdstrike_model.pkl")
joblib.dump(best_species_model, "xgb_species_model.pkl")
joblib.dump(best_phase_model, "xgb_phase_model.pkl")


['xgb_phase_model.pkl']

In [61]:
def create_user_input(date, timeofday, airport, visibility):
    date = pd.to_datetime(date)
    return pd.DataFrame([{
        "incident_year": date.year,
        "incident_month": date.month,
        "timeofday": timeofday,
        "airport": airport,
        "visibility": visibility
    }])


In [62]:
def compare_airports(date, timeofday, dep_airport, arr_airport, visibility):
    dep_input = create_user_input(date, timeofday, dep_airport, visibility)
    arr_input = create_user_input(date, timeofday, arr_airport, visibility)

    dep_prob = best_birdstrike_model.predict_proba(dep_input)[:, 1][0]
    arr_prob = best_birdstrike_model.predict_proba(arr_input)[:, 1][0]

    return {
        "departure_airport_risk": round(dep_prob, 3),
        "arrival_airport_risk": round(arr_prob, 3),
        "higher_risk_airport": dep_airport if dep_prob > arr_prob else arr_airport
    }


In [66]:
def birdstrike_prediction_system(
    date,
    timeofday,
    departure_airport,
    arrival_airport,
    visibility
):
   
    user_input = create_user_input(
        date, timeofday, departure_airport, visibility
    )

   
    strike_prob = best_birdstrike_model.predict_proba(user_input)[:, 1][0]

    
    species_encoded = best_species_model.predict(user_input)[0]
    species = species_encoder.inverse_transform([species_encoded])[0]

   
    phase_encoded = best_phase_model.predict(user_input)[0]
    phase = phase_encoder.inverse_transform([phase_encoded])[0]

    
    airport_risk = compare_airports(
        date, timeofday, departure_airport, arrival_airport, visibility
    )

    return {
        "birdstrike_probability": round(float(strike_prob), 3),
        "most_likely_bird_species": species,
        "most_likely_flight_phase": phase,
        "airport_risk_analysis": airport_risk
    }



In [67]:
birdstrike_prediction_system(
    date="2010-07-15",
    timeofday="Day",
    departure_airport="JFK Intl",
    arrival_airport="LAX Intl",
    visibility="Clear"
)


{'birdstrike_probability': 0.071,
 'most_likely_bird_species': 'Other',
 'most_likely_flight_phase': 'APPROACH',
 'airport_risk_analysis': {'departure_airport_risk': np.float32(0.071),
  'arrival_airport_risk': np.float32(0.071),
  'higher_risk_airport': 'LAX Intl'}}

In [68]:
birdstrike_prediction_system(
    date="2012-01-15",
    timeofday="Night",
    departure_airport="DEN Intl",
    arrival_airport="PHX Intl",
    visibility="Clear"
)


{'birdstrike_probability': 0.157,
 'most_likely_bird_species': 'Other',
 'most_likely_flight_phase': 'APPROACH',
 'airport_risk_analysis': {'departure_airport_risk': np.float32(0.157),
  'arrival_airport_risk': np.float32(0.157),
  'higher_risk_airport': 'PHX Intl'}}